In [0]:
from delta.tables import DeltaTable

In [0]:
%sql
CREATE TABLE workspace.stream.sourcetable
(
  color STRING
)
USING DELTA

In [0]:
#creates a managed table adding .location() would create an external table
DeltaTable.createIfNotExists(spark)\
    .tableName("workspace.stream.sourcetable")\
    .addColumn("color", "STRING")\
    .execute()

In [0]:
DeltaTable.createIfNotExists(spark)\
    .tableName("workspace.stream.sinktable")\
    .addColumn("color", "STRING")\
    .execute()

In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
INSERT INTO workspace.stream.sourcetable VALUES
('red'),
('green'),
('blue'),
('black'),
('red'),
('white'),
('orange'),
('yellow'),
('purple'),
('brown')

num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
SELECT * FROM workspace.stream.sourcetable

color
red
green
blue
black
white
orange
yellow
purple
brown
red


In [0]:
%sql
INSERT INTO workspace.stream.sourcetable VALUES
('gold'),
('gold'),
('white'),
('silver'),
('bronze'),
('brown')

num_affected_rows,num_inserted_rows
6,6


In [0]:
df = spark.readStream.table('workspace.stream.sourcetable')

In [0]:
df = df.groupBy("color").agg(count("*").alias("count"))

In [0]:
df.writeStream.format('delta')\
    .outputMode("complete")\
    .trigger(once=True)\
    .option("checkpointLocation", "/Volumes/workspace/stream/streaming/output/checkpoint")\
    .option("path", "/Volumes/workspace/stream/streaming/output/Data")\
    .start()

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/stream/streaming/output/Data`

color,count
orange,3
yellow,3
bronze,1
silver,1
purple,2
white,4
brown,4
green,2
black,2
gold,2
